Import libs

In [ ]:
import os #give access to OS functionality, like working with files and folder
import csv #Allows reading and writing CSV files
import numpy as np
print(np.__version__)
import scipy.io as sio
import sklearn
from sklearn.covariance import GraphicalLassoCV
print(sklearn.__version__)
import nilearn
print(nilearn.__version__)
from nilearn import connectome


Manage Files

In [45]:
# Output path
save_path = '/home/celery/Documents/Research/dataset/Outputs'

# Number of subjects
num_subjects = 1000

# Selected pipeline
pipeline = 'cpac'

# Files to fetch
derivatives = ['rois_ho']

# Get the root folder
root_folder = '/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho'


In [46]:
def get_ids(num_subjects=None, short=True):
    """
        num_subjects   : number of subject IDs to get
        short          : True of False, specifies whether to get short or long subject IDs (Eg: 51431 or NYU_0051431_session_1_rest_1)

    return:
        subject_IDs    : list of subject IDs (length num_subjects)
    """
    
    if short:
        subject_IDs = np.loadtxt('/home/celery/Documents/Research/dataset/valid_subject_ids.txt', dtype=int)
        subject_IDs = subject_IDs.astype(str)
    else:
        subject_IDs = np.loadtxt('/home/celery/Documents/Research/dataset/full_subject_ids.txt', dtype=str)

    if num_subjects is not None:
        subject_IDs = subject_IDs[:num_subjects]

    return subject_IDs

Testing after getting ids

In [ ]:

subject_IDs = get_ids(num_subjects=None, short=True)
print("Subject IDs:", subject_IDs)
print("Number of IDs:", len(subject_IDs))



In [48]:
def fetch_filenames(subject_list, file_type):
    """
        subject_list : list of short subject IDs in string format
        file_type    : must be one of the available file types

    returns:

        filenames    : list of filetypes (same length as subject_list)
    """

    # Specify file mappings for the possible file types
    filemapping = {'func_preproc': '_func_preproc.nii.gz',
                   'rois_aal': '_rois_aal.1D',
                   'rois_cc200': '_rois_cc200.1D',
                   'rois_ho': '_rois_ho.1D'}

    # The list to be filled
    filenames = []

    # Load subject ID lists
    subject_IDs = get_ids(short=True)
    subject_IDs = subject_IDs.tolist()
    full_IDs = get_ids(short=False)

    # Fill list with requested file paths
    for s in subject_list:
        try:
            if file_type in filemapping:
                idx = subject_IDs.index(s)
                pattern = full_IDs[idx] + filemapping[file_type]
            else:
                pattern = s + file_type
            

            filenames.append(os.path.join(root_folder, pattern))
        except ValueError:
            # Return N/A if subject ID is not found
            filenames.append('N/A')

    return filenames

Testing the function

In [49]:
test_subjects = ['50724', '51585']  
file_type = 'rois_ho'               

file_list = fetch_filenames(test_subjects, file_type)


In [50]:

for f in file_list:
    print(f)


/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/Leuven_2_0050724_rois_ho.1D
/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/SBL_0051585_rois_ho.1D


In [51]:
def find_subject_file(subject_id, folder_path, suffix='_rois_ho.1D'):
    """
    Search for a file in folder_path that matches the given subject_id.

    subject_id   : short subject ID like '50004'
    folder_path  : the path where .1D files are stored (e.g., rois_ho folder)
    suffix       : file suffix to look for (default is '_rois_ho.1D')

    Returns:
        file_path  : full path to the matching file, or 'N/A' if not found
    """
    for fname in os.listdir(folder_path):
        if subject_id in fname and fname.endswith(suffix):
            return os.path.join(folder_path, fname)
    return 'N/A'


Testing

In [52]:
folder_path = "/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/" # this function is different from the  code because my data is flattened
subject_ids = ['50004', '50005', '50006']

for sid in subject_ids:
    file_path = find_subject_file(sid, folder_path)
    print("Subject", sid, "->", file_path)


Subject 50004 -> /home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/Pitt_0050004_rois_ho.1D
Subject 50005 -> /home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/Pitt_0050005_rois_ho.1D
Subject 50006 -> /home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/Pitt_0050006_rois_ho.1D


In [33]:
'''def fetch_conn_matrices(subject_list, atlas_name, kind): #fetch mat file after subject_connectivity()
    """
        subject_list : list of short subject IDs in string format
        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200
        kind         : the kind of correlation used to estimate the matrices, i.e.

    returns:
        connectivity : list of square connectivity matrices, one for each subject in subject_list
    """

    conn_files = fetch_filenames(subject_list,
                                 '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')

    conn_matrices = []

    for fl in conn_files:
        print("Reading connectivity file %s" % fl)
        try:
            mat = sio.loadmat(fl)['connectivity']
            conn_matrices.append(mat)
        except IOError:
            print("File %s does not exist" % fl)

    return conn_matrices
'''


'def fetch_conn_matrices(subject_list, atlas_name, kind): #fetch mat file after subject_connectivity()\n    """\n        subject_list : list of short subject IDs in string format\n        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200\n        kind         : the kind of correlation used to estimate the matrices, i.e.\n\n    returns:\n        connectivity : list of square connectivity matrices, one for each subject in subject_list\n    """\n\n    conn_files = fetch_filenames(subject_list,\n                                 \'_\' + atlas_name + \'_\' + kind.replace(\' \', \'_\') + \'.mat\')\n\n    conn_matrices = []\n\n    for fl in conn_files:\n        print("Reading connectivity file %s" % fl)\n        try:\n            mat = sio.loadmat(fl)[\'connectivity\']\n            conn_matrices.append(mat)\n        except IOError:\n            print("File %s does not exist" % fl)\n\n    return conn_matrices\n'

In [53]:
def get_timeseries(subject_list, atlas_name):
    """
        subject_list : list of short subject IDs in string format
        atlas_name   : the atlas based on which the timeseries are generated e.g. aal, cc200

    returns:
        ts           : list of timeseries arrays, each of shape (timepoints x regions)
    """

    ts_files = fetch_filenames(subject_list, 'rois_' + atlas_name)

    ts = []

    for fl in ts_files:
        print("Reading timeseries file %s" % fl)
        ts.append(np.loadtxt(fl, skiprows=0))

    return ts

Function testing

In [54]:
#ts = get_timeseries(['50004'], 'ho') 


In [55]:
def norm_timeseries(ts_list):
    """
        ts_list    : list of timeseries arrays, each of shape (timepoints x regions)

    returns:
        norm_ts    : list of normalised timeseries arrays, same shape as ts_list
    """

    norm_ts = []

    for ts in ts_list:
        norm_ts.append(nilearn.signal.clean(ts, detrend=False))

    return norm_ts

In [37]:
def subject_connectivity(timeseries, subject, atlas_name, kind, save=True, save_path=root_folder):
    """
        timeseries   : timeseries table for subject (timepoints x regions)
        subject      : the subject short ID
        atlas_name   : name of the atlas used
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        save         : save the connectivity matrix to a file
        save_path    : specify path to save the matrix if different from subject folder

    returns:
        connectivity : connectivity matrix (regions x regions)
    """

    print("Estimating %s matrix for subject %s" % (kind, subject))

    if kind == 'lasso':
        # Graph Lasso estimator
        covariance_estimator = GraphicalLassoCV(verbose=1)
        covariance_estimator.fit(timeseries)
        connectivity = covariance_estimator.covariance_
        print('Covariance matrix has shape {0}.'.format(connectivity.shape))

    elif kind in ['tangent', 'partial correlation', 'correlation']:
        conn_measure = connectome.ConnectivityMeasure(kind=kind)
        connectivity = conn_measure.fit_transform([timeseries])[0]

    # defining custom save path
    save_path_mat = '/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/mat/'
    
    if save:
        subject_file = os.path.join(save_path_mat,
                            subject + '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')

        sio.savemat(subject_file, {'connectivity': connectivity})

    return connectivity


Function testing

In [38]:
# ts = np.loadtxt('/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/rois_ho/Caltech_0051473_rois_ho.1D')  # (timepoints x regions)
# conn = subject_connectivity(ts, '51473', 'ho', 'correlation') #uncomment to test


In [39]:
# mat_data = sio.loadmat('/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/mat/51473_ho_correlation.mat')
# print(mat_data.keys())
# print(mat_data['connectivity'])
# conn = mat_data['connectivity']
# print(conn.shape)

In [64]:
def group_connectivity(timeseries, subject_list, atlas_name, kind, save=True, save_path=root_folder): #batch version of the function above
    """
        timeseries   : list of timeseries tables for subjects (timepoints x regions)
        subject_list : the subject short IDs list
        atlas_name   : name of the atlas used
        kind         : the kind of connectivity to be used, e.g. lasso, partial correlation, correlation
        save         : save the connectivity matrix to a file
        save_path    : specify path to save the matrix if different from subject folder

    returns:
        connectivity : connectivity matrix (regions x regions)
    """

    if kind == 'lasso':
        # Graph Lasso estimator
        covariance_estimator = GraphicalLassoCV(verbose=1)
        connectivity_matrices = []

        for i, ts in enumerate(timeseries):
            covariance_estimator.fit(ts)
            connectivity = covariance_estimator.covariance_
            connectivity_matrices.append(connectivity)
            print('Covariance matrix has shape {0}.'.format(connectivity.shape))

    elif kind in ['tangent', 'partial correlation', 'correlation']:
        conn_measure = connectome.ConnectivityMeasure(kind=kind)
        connectivity_matrices = conn_measure.fit_transform(timeseries)
    # defining custom save path for .mat files
    save_path_mat = '/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/mat/'

    if save:
        for i, subject in enumerate(subject_list):
            subject_file = os.path.join(save_path_mat,
                                        subject + '_' + atlas_name + '_' + kind.replace(' ', '_') + '.mat')
            sio.savemat(subject_file, {'connectivity': connectivity_matrices[i]})
            print("Saving connectivity matrix to %s" % subject_file)

    return connectivity_matrices


Getting the connectivity from using group function above

In [58]:
subject_file = '/home/celery/Documents/Research/dataset/valid_subject_ids.txt'
subject_list = np.loadtxt(subject_file, dtype = str).tolist()

In [ ]:
ts_list = get_timeseries(subject_list, 'ho')

In [60]:
norm_ts_list = norm_timeseries(ts_list)

In [ ]:
group_connectivity(
    timeseries=ts_list,
    subject_list=subject_list,
    atlas_name='ho',
    kind='correlation',
    save=True,
    save_path= '/home/celery/Documents/Research/dataset/Outputs/cpac/filt_global/mat'
)